In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer
from tqdm import tqdm
import torch.nn.functional as F

### Utility function

In [2]:
def preprocessing(arguments, key_points, labels):
    columns = ['topic', 'premise', 'hypothesis', 'label']
    data = []
    for index, row in labels.iterrows():
        argument_row = arguments.loc[arguments['arg_id'] == row['arg_id']]
        topic = argument_row['topic'].iloc[0]
        premise = argument_row['argument'].iloc[0]
        kp_row = key_points.loc[key_points['key_point_id'] == row['key_point_id']]
        hypothesis = kp_row['key_point'].iloc[0]
        label = row['label']
        data.append([topic, premise, hypothesis, label])

    result = pd.DataFrame(data, columns=columns)
    return result

### Create Torch Dataset

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, text_df, tokenizer, max_length):
        self.premises = list(text_df['premise'].values+tokenizer.sep_token+text_df['topic'].values)
        self.hypotheses = list(text_df['hypothesis'].values+tokenizer.sep_token+text_df['topic'].values)
        self.labels = list(text_df['label'])
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.premises)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.premises[idx],
            self.hypotheses[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }
        return item


### Define the LightningModule

In [4]:
class KPM(pl.LightningModule):
    def __init__(self, model, learning_rate=2e-5, weight_decay=0.001):
        super(KPM, self).__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
        return outputs

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        outputs = self(input_ids, attention_mask, labels)
        print("--------------training-------------------------------")
        print("Shape: "+ str(outputs.logits.shape))
        print(outputs.logits)
        self.log("train_loss", torch.clone(outputs.loss).detach())
        one_hot_labels = torch.stack([1 - labels, labels], dim=-1)
        one_hot_labels = one_hot_labels.float()
        # loss = outputs.loss
        loss = F.binary_cross_entropy_with_logits(outputs.logits, one_hot_labels, reduction='mean')
        self.log("train_loss", torch.clone(loss).detach())
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", torch.clone(outputs.loss).detach())
        one_hot_labels = torch.stack([1 - labels, labels], dim=-1)
        one_hot_labels = one_hot_labels.float()
        # loss = outputs.loss
        loss = F.binary_cross_entropy_with_logits(outputs.logits, one_hot_labels, reduction='mean')
        self.log("val_loss", torch.clone(loss).detach())
        return  loss

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.learning_rate)
        return optimizer

### Import Dataset and Preprocessing

In [5]:
train_argument_df = pd.read_csv('./data/arguments_train.csv')
train_kp_df = pd.read_csv('./data/key_points_train.csv')
train_label_df = pd.read_csv('./data/labels_train.csv')
train_df = preprocessing(train_argument_df, train_kp_df, train_label_df)


val_argument_df = pd.read_csv('./data/arguments_dev.csv')
val_kp_df = pd.read_csv('./data/key_points_dev.csv')
val_label_df = pd.read_csv('./data/labels_dev.csv')
val_df = preprocessing(val_argument_df, val_kp_df, val_label_df)

test_argument_df = pd.read_csv('./data/arguments_test.csv')
test_kp_df = pd.read_csv('./data/key_points_test.csv')
test_label_df = pd.read_csv('./data/labels_test.csv')
test_df = preprocessing(test_argument_df, test_kp_df, test_label_df)

### Training phase

In [1]:
model_name = "cross-encoder/nli-distilroberta-base"
num_classes = 2
max_length = 512
batch_size = 16
learning_rate = 5e-05
weight_decay = 0.001

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_classes, ignore_mismatched_sizes = True)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
# model.classifier = torch.nn.Linear(model.config.hidden_size, num_classes)
# Change the loss function to binary cross-entropy

# Get the vocabulary
vocab = tokenizer.get_vocab()

# Check if the "CLS" token is present in the vocabulary
cls_token_present = tokenizer.cls_token in vocab
sep_token_present = tokenizer.sep_token in vocab
print("Is CLS token present in the vocabulary?", cls_token_present)
print("Is SEP token present in the vocabulary?", sep_token_present)

train_dataset = Dataset(train_df, tokenizer, max_length)
val_dataset = Dataset(val_df, tokenizer, max_length)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = KPM(model, learning_rate)

# Define a ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='./checkpoint',
    filename='nli_model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min'
)

early_stopping =EarlyStopping(
    monitor="val_loss",
    min_delta=0.01,
    patience=3
)

# define trainer
trainer = Trainer(
    min_epochs = 0, # change this
    max_epochs = 20,
    callbacks=[checkpoint_callback, early_stopping],
    accelerator="auto",
    #progress_bar_refresh_rate=30,
    #gpus = 1 if device.type == 'cuda' else 0
    devices = 1 if torch.cuda.is_available() else None
)

torch.cuda.empty_cache()

# start training
trainer.fit(model, train_loader, val_loader)
print("Training is finished")

NameError: name 'AutoTokenizer' is not defined

### Prediction and Evaluation

In [8]:
# Assuming you have already trained the model and saved a checkpoint
checkpoint_path = './checkpoint/nli_model-epoch=01-val_loss=0.37.ckpt'
# Load the model from the checkpoint
model_name = "cross-encoder/nli-distilroberta-base"
num_classes = 2
max_length = 512
batch_size = 16
learning_rate = 5e-05
weight_decay = 0.001

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_classes, ignore_mismatched_sizes = True)
loaded_model = KPM.load_from_checkpoint(model=model, checkpoint_path=checkpoint_path)
print("hello world")

# Set the model to evaluation mode
loaded_model.eval()

test_dataset = Dataset(test_df, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Make predictions on the test set with probabilities
all_predictions = []
all_probabilities = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting"):
        input_ids, attention_mask = batch['input_ids'], batch['attention_mask']
        outputs = loaded_model(input_ids, attention_mask)
        logits = outputs.logits
        # print(logits)
        probabilities = F.softmax(logits, dim=1)
        predictions = torch.argmax(logits, dim=1).tolist()
        # print(predictions)
        print(probabilities)
        all_predictions.extend(predictions)
        all_probabilities.extend(probabilities.tolist())


print(all_predictions)
print(all_probabilities)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cross-encoder/nli-distilroberta-base and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


hello world


Predicting:   0%|          | 1/215 [00:08<30:54,  8.66s/it]

tensor([[8.5138e-01, 1.4862e-01],
        [9.5304e-01, 4.6955e-02],
        [1.8656e-02, 9.8134e-01],
        [9.6801e-05, 9.9990e-01],
        [4.9789e-02, 9.5021e-01],
        [9.8562e-01, 1.4378e-02],
        [9.9964e-01, 3.6004e-04],
        [7.0038e-01, 2.9962e-01],
        [9.7845e-01, 2.1546e-02],
        [9.4093e-05, 9.9991e-01],
        [5.4960e-01, 4.5040e-01],
        [1.6561e-02, 9.8344e-01],
        [9.8758e-01, 1.2423e-02],
        [9.4357e-01, 5.6434e-02],
        [9.1618e-05, 9.9991e-01],
        [8.9708e-01, 1.0292e-01]])


Predicting:   1%|          | 2/215 [00:17<30:35,  8.62s/it]

...

tensor([[2.6431e-04, 9.9974e-01],
        [1.9602e-04, 9.9980e-01],
        [6.1879e-01, 3.8121e-01],
        [9.9773e-01, 2.2734e-03],
        [9.4752e-01, 5.2483e-02],
        [1.9436e-01, 8.0564e-01],
        [1.0774e-01, 8.9226e-01],
        [9.8363e-01, 1.6374e-02],
        [8.4544e-01, 1.5456e-01],
        [7.7038e-01, 2.2962e-01],
        [9.9126e-01, 8.7363e-03],
        [8.3547e-01, 1.6453e-01],
        [8.1612e-01, 1.8388e-01],
        [9.8382e-01, 1.6185e-02],
        [9.9318e-01, 6.8216e-03],
        [9.8075e-01, 1.9249e-02]])


Predicting: 100%|█████████▉| 214/215 [30:58<00:08,  8.51s/it]

tensor([[6.1331e-04, 9.9939e-01],
        [9.2636e-01, 7.3641e-02],
        [8.1997e-01, 1.8003e-01],
        [9.4873e-01, 5.1266e-02],
        [8.4854e-01, 1.5146e-01],
        [9.9468e-01, 5.3219e-03],
        [9.5679e-01, 4.3208e-02],
        [6.1372e-01, 3.8628e-01],
        [9.5266e-01, 4.7338e-02],
        [5.7470e-01, 4.2530e-01],
        [8.0666e-01, 1.9334e-01],
        [3.5936e-01, 6.4064e-01],
        [7.4850e-01, 2.5150e-01],
        [5.5690e-01, 4.4310e-01],
        [9.8293e-01, 1.7073e-02],
        [1.6950e-02, 9.8305e-01]])


Predicting: 100%|██████████| 215/215 [30:59<00:00,  8.65s/it]

tensor([[9.9872e-01, 1.2806e-03],
        [9.9987e-01, 1.3329e-04]])
[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 

In [7]:
print(len(all_predictions))
print(len(all_probabilities))

3426
3426


In [14]:
print(all_probabilities)
all_probabilities = [max(sublist) for sublist in all_probabilities]
print(all_probabilities)

[0.8513832688331604, 0.9530447125434875, 0.9813437461853027, 0.9999032020568848, 0.9502105712890625, 0.9856218695640564, 0.9996399879455566, 0.7003836631774902, 0.9784537553787231, 0.9999059438705444, 0.5496004223823547, 0.9834389090538025, 0.9875774383544922, 0.9435656070709229, 0.9999083280563354, 0.8970844745635986, 0.9932764768600464, 0.5377925038337708, 0.9752899408340454, 0.9779468178749084, 0.9998916387557983, 0.6552639603614807, 0.9617480635643005, 0.9989585876464844, 0.9997628331184387, 0.9690905809402466, 0.7799464464187622, 0.9968823194503784, 0.9996205568313599, 0.7533658742904663, 0.9707180857658386, 0.9991835951805115, 0.9999151229858398, 0.6591960191726685, 0.9929141998291016, 0.6223980188369751, 0.5727285146713257, 0.9057391285896301, 0.9987348914146423, 0.6909785270690918, 0.9980675578117371, 0.9997209906578064, 0.99972003698349, 0.9550588130950928, 0.7782137989997864, 0.5976061224937439, 0.9899438619613647, 0.6962425112724304, 0.7719510197639465, 0.811423122882843, 0.

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\jhluo\AppData\Local\Temp\ipykernel_9044\1938414908.py:2 in <cell line: 2>               │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\jhluo\\AppData\\Local\\Temp\\ipykernel_9044\\1938414908.py'                          │
│                                                                                                  │
│ C:\Users\jhluo\AppData\Local\Temp\ipykernel_9044\1938414908.py:2 in <listcomp>                   │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\jhluo\\AppData\\Local\\Temp\\ipykernel_9044\\1938414908.py'                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: 'float' object is not iterable

### Extract reslut to csv file

In [17]:
print(test_df.columns)

Index(['topic', 'premise', 'hypothesis', 'label'], dtype='object')


In [19]:
add_df = pd.DataFrame({
    'prediction': all_predictions,
    'probability':all_probabilities
})
result_df = pd.concat([test_df,add_df],axis=1)
print(result_df)

                                                  topic  \
0        Routine child vaccinations should be mandatory   
1        Routine child vaccinations should be mandatory   
2        Routine child vaccinations should be mandatory   
3        Routine child vaccinations should be mandatory   
4        Routine child vaccinations should be mandatory   
...                                                 ...   
3421  Social media platforms should be regulated by ...   
3422     Routine child vaccinations should be mandatory   
3423     Routine child vaccinations should be mandatory   
3424     Routine child vaccinations should be mandatory   
3425     Routine child vaccinations should be mandatory   

                                                premise  \
0      Routine childhood vaccinations  should be man...   
1      Routine childhood vaccinations  should be man...   
2      Routine childhood vaccinations  should be man...   
3      Routine childhood vaccinations  should be man...

In [20]:
result_df.to_csv('./result/KPM_official_test.csv', index=False)
print(f"DataFrame has been saved to {'./result/KPM_official_test.csv'}")

DataFrame has been saved to ./result/KPM_official_test.csv


In [13]:
# print(all_probabilities)
new_test_df = pd.read_csv('./result/KPM_official_test.csv')
print(new_test_df)

                                                  topic  \
0        Routine child vaccinations should be mandatory   
1        Routine child vaccinations should be mandatory   
2        Routine child vaccinations should be mandatory   
3        Routine child vaccinations should be mandatory   
4        Routine child vaccinations should be mandatory   
...                                                 ...   
3421  Social media platforms should be regulated by ...   
3422     Routine child vaccinations should be mandatory   
3423     Routine child vaccinations should be mandatory   
3424     Routine child vaccinations should be mandatory   
3425     Routine child vaccinations should be mandatory   

                                                premise  \
0      Routine childhood vaccinations  should be man...   
1      Routine childhood vaccinations  should be man...   
2      Routine childhood vaccinations  should be man...   
3      Routine childhood vaccinations  should be man...

In [16]:
new_test_df['logit'] = all_probabilities
print(new_test_df)
new_test_df.to_csv('./result/KPM_official_test.csv', index=False)
print(f"DataFrame has been saved to {'./result/KPM_official_test.csv'}")

                                                  topic  \
0        Routine child vaccinations should be mandatory   
1        Routine child vaccinations should be mandatory   
2        Routine child vaccinations should be mandatory   
3        Routine child vaccinations should be mandatory   
4        Routine child vaccinations should be mandatory   
...                                                 ...   
3421  Social media platforms should be regulated by ...   
3422     Routine child vaccinations should be mandatory   
3423     Routine child vaccinations should be mandatory   
3424     Routine child vaccinations should be mandatory   
3425     Routine child vaccinations should be mandatory   

                                                premise  \
0      Routine childhood vaccinations  should be man...   
1      Routine childhood vaccinations  should be man...   
2      Routine childhood vaccinations  should be man...   
3      Routine childhood vaccinations  should be man...

### Calculate mAP

In [17]:
df_test = pd.read_csv('./result/KPM_official_test.csv')
print(df_test.columns)
print(df_test)

Index(['topic', 'premise', 'hypothesis', 'label', 'prediction', 'probability',
       'logit'],
      dtype='object')
                                                  topic  \
0        Routine child vaccinations should be mandatory   
1        Routine child vaccinations should be mandatory   
2        Routine child vaccinations should be mandatory   
3        Routine child vaccinations should be mandatory   
4        Routine child vaccinations should be mandatory   
...                                                 ...   
3421  Social media platforms should be regulated by ...   
3422     Routine child vaccinations should be mandatory   
3423     Routine child vaccinations should be mandatory   
3424     Routine child vaccinations should be mandatory   
3425     Routine child vaccinations should be mandatory   

                                                premise  \
0      Routine childhood vaccinations  should be man...   
1      Routine childhood vaccinations  should be man...

In [18]:
df_sorted = df_test.sort_values(by='probability', ascending= False)
print(df_sorted)

                                                  topic  \
3177     Routine child vaccinations should be mandatory   
2537     Routine child vaccinations should be mandatory   
2962     Routine child vaccinations should be mandatory   
2630     Routine child vaccinations should be mandatory   
730      Routine child vaccinations should be mandatory   
...                                                 ...   
3007     Routine child vaccinations should be mandatory   
1892               The USA is a good country to live in   
1948  Social media platforms should be regulated by ...   
1560               The USA is a good country to live in   
2649               The USA is a good country to live in   

                                                premise  \
3177  the government should not regulate what parent...   
2537  child vaccinations is not mandatory because it...   
2962  parents should have the freedom to decide what...   
2630  everyone should be free to choose what they wa...

In [19]:
top_50_df = df_sorted.head(int(0.5*len(df_sorted)))
print(top_50_df)

                                                  topic  \
3177     Routine child vaccinations should be mandatory   
2537     Routine child vaccinations should be mandatory   
2962     Routine child vaccinations should be mandatory   
2630     Routine child vaccinations should be mandatory   
730      Routine child vaccinations should be mandatory   
...                                                 ...   
2451  Social media platforms should be regulated by ...   
654                The USA is a good country to live in   
1749               The USA is a good country to live in   
1366  Social media platforms should be regulated by ...   
2358     Routine child vaccinations should be mandatory   

                                                premise  \
3177  the government should not regulate what parent...   
2537  child vaccinations is not mandatory because it...   
2962  parents should have the freedom to decide what...   
2630  everyone should be free to choose what they wa...

In [12]:
from sklearn.metrics import average_precision_score
import numpy as np

# Assuming A and B are the predicted and true labels for each instance, respectively
A = [0, 1, 0]
B = [1, 0, 1]

# Calculate Average Precision
ap = average_precision_score(label, prediction)

map_score = np.mean(ap)

print(f'Mean Average Precision (mAP): {map_score:.2f}')

Mean Average Precision (mAP): 0.65


### Make Test Dataset

In [30]:
argument_df = pd.read_csv('./data/arguments_test.csv')
print(argument_df.columns)

Index(['arg_id', 'argument', 'topic', 'stance'], dtype='object')


In [31]:
kp_df = pd.read_csv('./data/key_points_test.csv')
print(kp_df.columns)

Index(['key_point_id', 'key_point', 'topic', 'stance'], dtype='object')


In [37]:
test_data = []
for index,row in argument_df.iterrows():
    filtered_kp = kp_df[(kp_df['topic'] == row['topic']) & (kp_df['stance'] == row['stance'])]
    filtered_kp_list = filtered_kp.values.tolist()
    for line in filtered_kp_list:
        test_data.append([row['topic'],row['stance'],row['arg_id'],row['argument'],line[0],line[1]])
print(len(test_data))

header = ['topic','stance','arg_id','argument','key_point_id','key_point']
df = pd.DataFrame(test_data, columns=header)
print(df)
print(df.columns)



3923
                                               topic  stance     arg_id  \
0     Routine child vaccinations should be mandatory      -1    arg_0_0   
1     Routine child vaccinations should be mandatory      -1    arg_0_0   
2     Routine child vaccinations should be mandatory      -1    arg_0_0   
3     Routine child vaccinations should be mandatory      -1    arg_0_0   
4     Routine child vaccinations should be mandatory      -1    arg_0_1   
...                                              ...     ...        ...   
3918            The USA is a good country to live in       1  arg_2_209   
3919            The USA is a good country to live in       1  arg_2_209   
3920            The USA is a good country to live in       1  arg_2_209   
3921            The USA is a good country to live in       1  arg_2_209   
3922            The USA is a good country to live in       1  arg_2_209   

                                               argument key_point_id  \
0     Routine child va

In [36]:
df.to_csv('./data/KPM_test_data.csv', index=False)
print(f"DataFrame has been saved to {'./data/KPM_test_data.csv'}")

DataFrame has been saved to ./data/KPM_test_data.csv


### Prediction

In [8]:
#Add new header confidence_score
df = pd.read_csv('./data/KPM_test_data.csv')
print(df.columns)

Index(['Unnamed: 0.1', 'Unnamed: 0', 'topic', 'stance', 'arg_id', 'argument',
       'key_point_id', 'key_point', 'confidence_score'],
      dtype='object')


In [9]:
# df['confidence_score'] = ''
# df.to_csv('./data/KPM_test_data.csv')
# print(f"DataFrame has been saved to {'./data/KPM_test_data.csv'}")

In [11]:
# Assuming you have already trained the model and saved a checkpoint
checkpoint_path = './checkpoint/nli_model-epoch=01-val_loss=0.37.ckpt'
# Load the model from the checkpoint
model_name = "cross-encoder/nli-distilroberta-base"
num_classes = 2
max_length = 512
batch_size = 16
learning_rate = 5e-05
weight_decay = 0.001

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_classes, ignore_mismatched_sizes = True)
loaded_model = KPM.load_from_checkpoint(model=model, checkpoint_path=checkpoint_path)
print("hello world")

# Set the model to evaluation mode
loaded_model.eval()


# Make predictions on the test set with probabilities

all_probabilities = []
df = pd.read_csv('./data/KPM_test_data.csv')
for index, row in df.iloc[:2].iterrows():
    premise = row['argument']+tokenizer.sep_token+row['topic']
    hypothesis = row['key_point']+tokenizer.sep_token+row['topic']
    encoding = tokenizer(
            premise,
            hypothesis,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
    with torch.no_grad():
        logits = loaded_model(**encoding).logits
        print(F.softmax(logits, dim=1).tolist()[0])
        probabilities = F.softmax(logits, dim=1).tolist()[0][0]
    print(str(index)+": "+str(probabilities))
    df.at[index,'confidence_score'] = probabilities
    # all_probabilities.append(probabilities)

df.to_csv('./data/KPM_test_data.csv', index=False)
print("Add new data successfully!!!!!")
# print("Finished")
# print(all_probabilities)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cross-encoder/nli-distilroberta-base and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


hello world
[0.04978947713971138, 0.9502105712890625]
0: 0.04978947713971138
[0.9856218695640564, 0.01437812577933073]
1: 0.9856218695640564


### Make output json file for automatically evaluating

In [2]:
argument_df = pd.read_csv('./data/arguments_test.csv')
print(len(argument_df))
print(argument_df.columns)

723
Index(['arg_id', 'argument', 'topic', 'stance'], dtype='object')


In [3]:
test_df = pd.read_csv('./data/KPM_test_data.csv')
# test_df['confidence_score'] = 1- test_df['confidence_score']
print(len(test_df))
print(test_df)

3923
                                               topic  stance     arg_id  \
0     Routine child vaccinations should be mandatory      -1    arg_0_0   
1     Routine child vaccinations should be mandatory      -1    arg_0_0   
2     Routine child vaccinations should be mandatory      -1    arg_0_0   
3     Routine child vaccinations should be mandatory      -1    arg_0_0   
4     Routine child vaccinations should be mandatory      -1    arg_0_1   
...                                              ...     ...        ...   
3918            The USA is a good country to live in       1  arg_2_209   
3919            The USA is a good country to live in       1  arg_2_209   
3920            The USA is a good country to live in       1  arg_2_209   
3921            The USA is a good country to live in       1  arg_2_209   
3922            The USA is a good country to live in       1  arg_2_209   

                                               argument key_point_id  \
0     Routine child va

In [20]:
# test_df = test_df.drop(test_df.columns[0], axis=1)
# print(test_df)

                                               topic  stance     arg_id  \
0     Routine child vaccinations should be mandatory      -1    arg_0_0   
1     Routine child vaccinations should be mandatory      -1    arg_0_0   
2     Routine child vaccinations should be mandatory      -1    arg_0_0   
3     Routine child vaccinations should be mandatory      -1    arg_0_0   
4     Routine child vaccinations should be mandatory      -1    arg_0_1   
...                                              ...     ...        ...   
3918            The USA is a good country to live in       1  arg_2_209   
3919            The USA is a good country to live in       1  arg_2_209   
3920            The USA is a good country to live in       1  arg_2_209   
3921            The USA is a good country to live in       1  arg_2_209   
3922            The USA is a good country to live in       1  arg_2_209   

                                               argument key_point_id  \
0     Routine child vaccina

In [23]:
# test_df.to_csv('./data/KPM_test_data.csv', index=False)
# print("Add new data successfully!!!!!")

Add new data successfully!!!!!


In [24]:
output_data = []
arg_id = argument_df['arg_id'].values.tolist()
for arg in arg_id:
    filtered_df = test_df[test_df['arg_id'] == arg]
    kp_id = filtered_df['key_point_id'].values.tolist()
    score = filtered_df['confidence_score'].values.tolist()
    result_dict = dict(zip(kp_id, score))
    output_data.append(result_dict)
final_result = dict(zip(arg_id,output_data))
print(final_result)

{'arg_0_0': {'kp_0_0': 0.950210523, 'kp_0_1': 0.014378129999999989, 'kp_0_2': 0.00036001200000002065, 'kp_0_3': 0.824956879}, 'arg_0_1': {'kp_0_0': 0.961748116, 'kp_0_1': 0.0010414119999999638, 'kp_0_2': 0.00023716699999998259, 'kp_0_3': 0.344736099}, 'arg_0_2': {'kp_0_0': 0.779946506, 'kp_0_1': 0.0031176809999999833, 'kp_0_2': 0.00037944300000003484, 'kp_0_3': 0.969090639}, 'arg_0_3': {'kp_0_0': 0.970718089, 'kp_0_1': 0.00081640500000002, 'kp_0_2': 8.487700000003873e-05, 'kp_0_3': 0.753365889}, 'arg_0_4': {'kp_0_0': 0.997252888, 'kp_0_1': 0.025554359000000026, 'kp_0_2': 0.006825387000000016, 'kp_0_3': 0.828299209}, 'arg_0_5': {'kp_0_0': 0.03237760099999998, 'kp_0_1': 0.10669434099999997, 'kp_0_2': 0.023948430999999992, 'kp_0_3': 0.013217926000000046}, 'arg_0_6': {'kp_0_0': 0.99965146, 'kp_0_1': 0.0014120939999999749, 'kp_0_2': 0.00013506399999996255, 'kp_0_3': 0.99885866}, 'arg_0_7': {'kp_0_0': 0.999903, 'kp_0_1': 0.0002350209999999464, 'kp_0_2': 7.629399999997677e-05, 'kp_0_3': 0.470

In [25]:
import json
file_path = './result/test.json'
with open(file_path, 'w') as json_file:
    json.dump(final_result, json_file)
print(f"JSON file '{file_path}' created successfully.")

JSON file './result/test.json' created successfully.
